# Read Config File

In [1]:
import configparser as CP
import pathlib as PL
import subprocess as SP

In [2]:
my_config = CP.ConfigParser(interpolation=CP.ExtendedInterpolation())
my_config.read("read_config.cfg")

['read_config.cfg']

## Sections

Reading the sections available on the config file.

In [3]:
my_config.sections()

['PROJECT', 'BIN', 'OUT']

Checking if one specific section is available on the file:

In [4]:
section_name = "OUT"
if my_config.has_section(section_name):
    print(f"There is a section name '{section_name}'")
else:
    print(f"Sorry, no section '{section_name}' in the file")

There is a section name 'OUT'


## Values

There are two ways to read the values of a section: as function or as dictiornary:

In [5]:
print(f"root dir: '{my_config.get('DEFAULT', 'root_dir')}'")
print(f"root dir: '{my_config['DEFAULT']['root_dir']}'")

root dir: 'cwd'
root dir: 'cwd'


It's interesting to note that the values from the `DEFAULT` section are "incorporated" to the other sections too:

In [6]:
for key in my_config['OUT']:
    print(key)

out_dir
out_name
format
root_dir


We can use the value of the `root_dir` to set the directory with the scripts, and the directory for the output

In [7]:
if my_config['DEFAULT']["root_dir"] == "cwd":
    root_dir =  PL.Path.cwd()
else:
    root_dir = PL.Path(my_config['DEFAULT']["root_dir"])
run_dir = root_dir.joinpath(my_config['BIN']['bin_dir'])
out_dir = root_dir.joinpath(my_config['OUT']['out_dir'])

After setting the variable `out_dir`, we check if the directory actually exist, and if not, then we need to create one.

In [8]:
# print(f"Output directory: '{out_dir}'")
try:
    if not out_dir.exists():
        print(f"Output directory doesn't exist. Creating one.")
        out_dir.mkdir()
except Exception as ee:
    print(f"Error '{ee}' while creating '{out_dir}'")

Output directory doesn't exist. Creating one.


## Running a Script

After defining the bin and output directories we run the script. To make the script generic, we are quoting `my_script` because in Windows there can be a space in the path, like `C:\Program Files`, which crashes the script.

In [9]:
my_script = run_dir.joinpath(my_config['BIN']['exe_name'])
run_cmd = f"python \"{my_script}\""

In [11]:
result = SP.run(run_cmd, stdout=SP. PIPE, stderr=SP.STDOUT, text=True, shell=True)
result.check_returncode()
print(f"Result: '{result.stdout}'")

Result: 'Hello world
'


## Saving Output

Saving the output of the script

In [ ]:
my_output = out_dir.joinpath(my_config['OUT']['out_name'])
with open(my_output, 'w') as fout:
    fout.writelines(result.stdout)